**title.basics.tsv.gz**
- tconst (string) - alphanumeric unique identifier of the title
- titleType (string) - the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
- primaryTitle (string) - the more popular title / the title used by the filmmakers on promotional materials at the point of release
- startYear (YYYY) - represents the release year of a title. In the case of TV Series, it is the series start year
- runtimeMinutes - primary runtime of the title, in minutes
- genres (string array) - includes up to three genres associated with the title

**title.crew.tsv.gz**
- tconst (string) - alphanumeric unique identifier of the title
- directors (array of nconsts) - director(s) of the given title
- writers (array of nconsts) - writer(s) of the given title

**title.ratings.tsv.gz**
- tconst (string) - alphanumeric unique identifier of the title
- averageRating - weighted average of all the individual user ratings

**name.basics.tsv.gz**
- nconst (string) - alphanumeric unique identifier of the name/person
- primaryName (string) - name by which the person is most often credited

**title.principals.tsv.gz**
- tconst (string) - alphanumeric unique identifier of the title
- nconst (string) - alphanumeric unique identifier of the name/person
- category (string) - the category of job that person was in
- job (string) - the specific job title if applicable, else '\N'

# Setup

Pip installs

In [1]:
!pip install pyspark


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Python imports

In [2]:
import os.path as path
from pyspark.sql import SparkSession
from pyspark.sql import column
from pyspark.sql.functions import *
import matplotlib.pyplot as plt

In [5]:
title_basics_dataset = "title.basics.tsv"

title_crew_dataset = "title.crew.tsv"

title_ratings_dataset = "title.ratings.tsv"

name_basics_dataset = "name.basics.tsv"

title_principals_dataset = "title.principals.tsv"

if not(path.isfile(title_basics_dataset)):
  !wget https://datasets.imdbws.com/title.basics.tsv.gz
  !gzip -d title.basics.tsv.gz

if not(path.isfile(title_crew_dataset)):
  !wget https://datasets.imdbws.com/title.crew.tsv.gz
  !gzip -d title.crew.tsv.gz

if not(path.isfile(title_ratings_dataset)):
  !wget https://datasets.imdbws.com/title.ratings.tsv.gz
  !gzip -d title.ratings.tsv.gz

if not(path.isfile(name_basics_dataset)):
  !wget https://datasets.imdbws.com/name.basics.tsv.gz
  !gzip -d name.basics.tsv.gz

if not(path.isfile(title_principals_dataset)):
  !wget https://datasets.imdbws.com/title.principals.tsv.gz
  !gzip -d title.principals.tsv.gz

'wget' is not recognized as an internal or external command,
operable program or batch file.
'gzip' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'gzip' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'gzip' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'gzip' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'gzip' is not recognized as an internal or external command,
operable program or batch file.


# Preparing Dataset

Helpers

In [4]:
def init_spark():
  return SparkSession \
        .builder \
        .appName("Spark") \
        .config("spark.executor.memory", "8g") \
        .config("spark.driver.memory", "8g") \
        .getOrCreate()

spark = init_spark()


Create base dataframes

In [5]:
spark

In [5]:
title_basics_df = spark.read.option("delimiter", "\t").option("header", True).csv(title_basics_dataset)

title_crew_df = spark.read.option("delimiter", "\t").option("header", True).csv(title_crew_dataset)

title_ratings_df = spark.read.option("delimiter", "\t").option("header", True).csv(title_ratings_dataset)

name_basics_df = spark.read.option("delimiter", "\t").option("header", True).csv(name_basics_dataset)

title_principals_df = spark.read.option("delimiter", "\t").option("header", True).csv(title_principals_dataset)

NameError: name 'title_basics_dataset' is not defined

In [ ]:
title_basics_df.show()
title_crew_df.show()
title_ratings_df.show()
name_basics_df.show()
title_principals_df.show()

## Filters

Filter entries for only movie entries

In [ ]:
title_basics_df = title_basics_df.filter(title_basics_df.titleType == "movie")
#title_basics_df.show()

Remove entries where the genres is null (\N)

In [ ]:
title_basics_df = title_basics_df.filter(title_basics_df.genres != "\\N")
#title_basics_df.show()

## Integration

Separate genres and keep single genre

Cases:

| In 0 | In 1 | In 2 |
|------|------|------|
| T    | T    | T    |
| T    | T    | F    |
| T    | F    | T    |
| F    | T    | T    |
| T    | T    | F    |
| T    | F    | T    |
| F    | T    | T    |
| F    | F    | F    |

In [ ]:
#When there are multiple genres:
#if only 1 from the list is in the genre column -> pick genre from list
#if multiple genres from the list is in the genre column -> weigh by count?
drama_entry_count = 0
comedy_entry_count = 0
horror_entry_count = 0
romance_entry_count = 0
action_entry_count = 0
thriller_entry_count = 0

genre_column = split(title_basics_df["genres"], ",")

genres_to_keep = ["Drama", "Comedy", "Horror", "Thriller", "Romance", "Action"]

test = title_basics_df.filter(genre_column.getItem(0).isin(genres_to_keep))
#test = title_basics_df.filter(genre_column.getItem(0).isin(genres_to_keep))
test = test.withColumn("genre", genre_column.getItem(0))

test2 = title_basics_df.filter((genre_column.getItem(1).isin(genres_to_keep)) & ~(genre_column.getItem(0).isin(genres_to_keep) ))
#test2 = title_basics_df.filter(genre_column.getItem(1).isin(genres_to_keep))
test2 = test2.withColumn("genre", genre_column.getItem(1))

test3 = title_basics_df.filter((genre_column.getItem(2).isin(genres_to_keep)) & ~(genre_column.getItem(0).isin(genres_to_keep) | genre_column.getItem(1).isin(genres_to_keep)))
#test3 = title_basics_df.filter(genre_column.getItem(2).isin(genres_to_keep))
test3 = test3.withColumn("genre", genre_column.getItem(2))

title_basics_df = test.union(test2).union(test3)
#title_basics_df.show()

'''
#title_basics_df.groupBy('genres').count().orderBy(desc("count")).show()

intermediate_df = title_basics_df.withColumn("genre1", genre_column.getItem(0))

genre_rows1 = intermediate_df.where((intermediate_df.genre1 == "Drama") | (intermediate_df.genre1 == "Comedy") | (intermediate_df.genre1 == "Horror") | (intermediate_df.genre1 == "Thriller") | (intermediate_df.genre1 == "Romance") | (intermediate_df.genre1 == "Action")).groupBy('genre1').count().orderBy("genre1").take(6)

intermediate_df = title_basics_df.withColumn("genre2", genre_column.getItem(1))

genre_rows2 = intermediate_df.where((intermediate_df.genre2 == "Drama") | (intermediate_df.genre2 == "Comedy") | (intermediate_df.genre2 == "Horror") | (intermediate_df.genre2 == "Thriller") | (intermediate_df.genre2 == "Romance") | (intermediate_df.genre2 == "Action")).groupBy('genre2').count().orderBy("genre2").take(6)

intermediate_df = title_basics_df.withColumn("genre3", genre_column.getItem(2))

genre_rows3 = intermediate_df.where((intermediate_df.genre3 == "Drama") | (intermediate_df.genre3 == "Comedy") | (intermediate_df.genre3 == "Horror") | (intermediate_df.genre3 == "Thriller") | (intermediate_df.genre3 == "Romance") | (intermediate_df.genre3 == "Action")).groupBy('genre3').count().orderBy("genre3").take(6)

for row in genre_rows1:
  if (row["genre1"] == "Action"):
    action_entry_count = action_entry_count + row["count"]

  if (row["genre1"] == "Comedy"):
    comedy_entry_count = comedy_entry_count + row["count"]

  if (row["genre1"] == "Drama"):
    drama_entry_count = drama_entry_count + row["count"]

  if (row["genre1"] == "Horror"):
    horror_entry_count = horror_entry_count + row["count"]

  if (row["genre1"] == "Romance"):
    romance_entry_count = romance_entry_count + row["count"]

  if (row["genre1"] == "Thriller"):
    thriller_entry_count = thriller_entry_count + row["count"]

for row in genre_rows2:
  if (row["genre2"] == "Action"):
    action_entry_count = action_entry_count + row["count"]

  if (row["genre2"] == "Comedy"):
    comedy_entry_count = comedy_entry_count + row["count"]

  if (row["genre2"] == "Drama"):
    drama_entry_count = drama_entry_count + row["count"]

  if (row["genre2"] == "Horror"):
    horror_entry_count = horror_entry_count + row["count"]

  if (row["genre2"] == "Romance"):
    romance_entry_count = romance_entry_count + row["count"]

  if (row["genre2"] == "Thriller"):
    thriller_entry_count = thriller_entry_count + row["count"]

for row in genre_rows3:
  if (row["genre3"] == "Action"):
    action_entry_count = action_entry_count + row["count"]

  if (row["genre3"] == "Comedy"):
    comedy_entry_count = comedy_entry_count + row["count"]

  if (row["genre3"] == "Drama"):
    drama_entry_count = drama_entry_count + row["count"]

  if (row["genre3"] == "Horror"):
    horror_entry_count = horror_entry_count + row["count"]

  if (row["genre3"] == "Romance"):
    romance_entry_count = romance_entry_count + row["count"]

  if (row["genre3"] == "Thriller"):
    thriller_entry_count = thriller_entry_count + row["count"]

genre_count_dic = {"Action": action_entry_count, "Comedy": comedy_entry_count, "Drama": drama_entry_count, "Horror": horror_entry_count, "Romance": romance_entry_count, "Thriller": thriller_entry_count}
sorted_genre_count_dic = dict(sorted(genre_count_dic.items(), key=lambda item: item[1], reverse=True))
print(sorted_genre_count_dic)

genre_column = split(title_basics_df["genres"], ",")

title_basics_df = title_basics_df.withColumn("genre", genre_column.getItem(0) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Drama") == 0 else \
       genre_column.getItem(0) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Comedy") == 0 else \
       genre_column.getItem(0) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Action") == 0 else \
       genre_column.getItem(0) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Romance") == 0 else \
       genre_column.getItem(0) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Thriller") == 0 else \
       genre_column.getItem(0) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Horror") == 0 else \
       genre_column.getItem(1) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Drama") == 1 else \
       genre_column.getItem(1) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Comedy") == 1 else \
       genre_column.getItem(1) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Action") == 1 else \
       genre_column.getItem(1) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Romance") == 1 else \
       genre_column.getItem(1) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Thriller") == 1 else \
       genre_column.getItem(1) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Horror") == 1 else \
       genre_column.getItem(2) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Drama") == 2 else \
       genre_column.getItem(2) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Comedy") == 2 else \
       genre_column.getItem(2) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Action") == 2 else \
       genre_column.getItem(2) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Romance") == 2 else \
       genre_column.getItem(2) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Thriller") == 2 else \
       genre_column.getItem(2) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Horror") == 2 else None)
'''

In [ ]:
title_crew_df.show()

In [ ]:
director_column = split(title_crew_df["directors"], ",")

Separate directors and keep single director

In [ ]:
director_column = split(title_crew_df["directors"], ",")
title_crew_df = title_crew_df.withColumn("director", director_column.getItem(0))
#title_crew_df.show()
title_crew_df = title_crew_df.drop("directors")
#title_crew_df.show()

In [ ]:
title_crew_df.show()

Separate writers and keep single writer

In [ ]:
writer_column = split(title_crew_df["writers"], ",")
title_crew_df = title_crew_df.withColumn("writer", writer_column.getItem(0))
#title_crew_df.show()
title_crew_df = title_crew_df.drop("writers")
#title_crew_df.show()

Get director names

In [ ]:
director_df = title_crew_df.select("director", "tconst")
renamed_director_df = director_df.withColumnRenamed("director", "nconst")

director_id_name_df = renamed_director_df.join(name_basics_df, on="nconst").select("tconst", "primaryName")
director_id_name_df = director_id_name_df.withColumnRenamed("primaryName", "directorPrimaryName")
#director_id_name_df.show()

Get writer names

In [ ]:
writer_df = title_crew_df.select("writer", "tconst")
renamed_writer_df = writer_df.withColumnRenamed("writer", "nconst")

writer_id_name_df = renamed_writer_df.join(name_basics_df, on="nconst").select("tconst", "primaryName")
writer_id_name_df = writer_id_name_df.withColumnRenamed("primaryName", "writerPrimaryName")
#writer_id_name_df.show()

In [ ]:
title_basics_and_title_crew_and_title_ratings_df = title_basics_df.join(title_ratings_df, on="tconst").join(director_id_name_df, on="tconst").join(writer_id_name_df, on="tconst")
#title_basics_and_title_crew_and_title_ratings_df.show()

In [ ]:
title_basics_and_title_crew_and_title_ratings_df.columns

Get actor names

In [ ]:
person_df = title_principals_df.select("tconst", "nconst", "category")
#person_df.show()

actor_df = person_df.filter(person_df.category == "actor")
#person_df = person_df.filter((person_df.category == "self") | (person_df.category == "cinematographer") | (person_df.category == "composer") | (person_df.category == "producer") | (person_df.category == "editor") | (person_df.category == "actor"))
#actor_df.show()

actor_df = actor_df.join(name_basics_df, on="nconst").select("tconst", "primaryName")
actor_id_name_df = actor_df.withColumnRenamed("primaryName", "actorPrimaryName")
#actor_id_name_df.show()
title_basics_and_title_crew_and_title_ratings_actor_df = actor_id_name_df.groupBy("tconst").agg(collect_list('actorPrimaryName').alias("actors"))
title_basics_and_title_crew_and_title_ratings_actor_df = title_basics_and_title_crew_and_title_ratings_actor_df.withColumn("actor1", title_basics_and_title_crew_and_title_ratings_actor_df.actors[0])
title_basics_and_title_crew_and_title_ratings_actor_df = title_basics_and_title_crew_and_title_ratings_actor_df.withColumn("actor2", title_basics_and_title_crew_and_title_ratings_actor_df.actors[1])
title_basics_and_title_crew_and_title_ratings_actor_df = title_basics_and_title_crew_and_title_ratings_actor_df.withColumn("actor3", title_basics_and_title_crew_and_title_ratings_actor_df.actors[2])
title_basics_and_title_crew_and_title_ratings_actor_df = title_basics_and_title_crew_and_title_ratings_actor_df.withColumn("actor4", title_basics_and_title_crew_and_title_ratings_actor_df.actors[3])
title_basics_and_title_crew_and_title_ratings_actor_df = title_basics_and_title_crew_and_title_ratings_df.join(actor_id_name_df, on="tconst")
#.groupBy("actorPrimaryName").pivot("tconst").agg(first("actorPrimaryName"))
#(nth_value("actorPrimaryName", 0).alias("actor1"), nth_value("actorPrimaryName", 1).alias("actor2"), nth_value("actorPrimaryName", 2).alias("actor3"), nth_value("actorPrimaryName", 3).alias("actor4"))
#test.show()
#title_basics_and_title_crew_and_title_ratings_actor_df = title_basics_and_title_crew_and_title_ratings_df.join(actor_id_name_df, on="tconst")
#title_basics_and_title_crew_and_title_ratings_actor_df.show()

In [ ]:
title_basics_and_title_crew_and_title_ratings_actor_df.columns

In [ ]:
final_df = title_basics_and_title_crew_and_title_ratings_actor_df.drop('titleType', 'originalTitle', 'isAdult', 'endYear', 'genres', 'numVotes', 'tconst', "actors")
final_df = final_df.withColumnRenamed('primaryTitle', 'title')
final_df = final_df.withColumnRenamed('startYear', 'year')
final_df = final_df.withColumnRenamed('runtimeMinutes', 'runtime')
final_df = final_df.withColumnRenamed('averageRating', 'rating')
final_df = final_df.withColumnRenamed('directorPrimaryName', 'director')
final_df = final_df.withColumnRenamed('writerPrimaryName', 'writer')
# final_df.show()
# final_df.persist()
# final_df.repartition(1).write.option("header", "true").save("mydata.csv")


In [ ]:
# movies releaed after 2021 to reduce data
final_df=final_df.filter(final_df.year.cast("int") > 2021)

In [ ]:
title_basics_and_title_crew_and_title_ratings_actor_df

In [ ]:
final_df.columns

In [ ]:
# final_df_sample = final_df.sample(fraction=0.1, seed=42)  # Sample 10% of the data
# final_df_sample.show(2)

In [ ]:
final_df.write.option("header", "true").csv("final_dataset.csv")